## 3. Use-cases and preparations for the petabyte scale

**Requirement:** First run #1

In [1]:
#From #2
import xarray as xr
port=9010
hostname=!echo $HOSTNAME
hosturl="https://"+hostname[0]+":"+str(port)
dsname="example"
zarr_url='/'.join([hosturl,"datasets",dsname,"zarr"])
storage_options=dict(verify_ssl=False)
ds=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options,
)

In [2]:
ds

<xarray.Dataset> Size: 314MB
Dimensions:    (lat: 192, time: 1032, bnds: 2, lon: 384)
Coordinates:
    height     float64 8B ...
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 3MB dask.array<chunksize=(60, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 6MB dask.array<chunksize=(60, 384, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] 17kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

### Server-side processing: Animated interactive geoplots

We build an API based on the pyviz stack that creates large figures of animated interactive geoplots for any selection of dataset, variable and coordinates of the hosted datasets. Additionally, you can set *kind* and *colorbar*. 

**Note**: On long term, plot creation should move to client-side. All that can be done in Jupyterlite.

In the following example, we write the plot into "test.html" which you can open in jupyterhub afterwards or download to your local computer.

In [2]:
varname="tas"
kind="image"
colorbar="coolwarm"
selection="time/-12_-1"
plotfile="test.html"
plot_url='/'.join([hosturl,"datasets",dsname,"plot",varname,kind,colorbar,selection])
print(plot_url)
!curl -k {plot_url} >{plotfile}

https://l40038.lvt.dkrz.de:9010/datasets/example/plot/tas/image/coolwarm/time/-12_-1
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5925k    0 5925k    0     0   318k      0 --:--:--  0:00:18 --:--:-- 1345k


### Server-side processing: Compression

In case bandwidth is the bottle-neck for users, we can help them with providing some server-side computing resources to e.g. further lossy compress our data. In the following, we firstly store the first 5 years on disk as provided. Secondly, we restart the app with enabled lossy compression and do the same. Afterwards, we compare speed and sizes.

In [3]:
import numpy as np
import time

def prepare_for_storage(ds_to_store):
    bnds=["lat_bnds","lon_bnds"]
    for a in bnds:
        ds_to_store[a]=ds_to_store[a].isel(time=0).squeeze()
    ds_to_store=ds_to_store.set_coords(bnds)
    ds_to_store["time_bnds"].load()

    return ds_to_store

def store_ds(ds_to_store,fn):
    !rm {fn}
    s=time.time()
    ds_to_store.to_netcdf(
        fn,
        unlimited_dims=["time"],
        encoding=dict(tas=dict(compression="zlib"))
    )
    e=time.time()
    print(e-s, " seconds")
    !ls -lrths {fn}

In [4]:
ds5=ds.where(ds.time.dt.year.isin(range(2015,2020)),drop=True)
ds5=prepare_for_storage(ds5)

In [5]:
store_ds(ds5,"compressed.nc")

2.4420695304870605  seconds
512 -rw-r--r-- 1 k204210 bm0021 6.6M Jan  8 10:42 compressed.nc


In [6]:
import xarray as xr
xr.open_dataset("compressed.nc").load()

<xarray.Dataset> Size: 18MB
Dimensions:    (lat: 192, bnds: 2, lon: 384, time: 60)
Coordinates:
    lat_bnds   (lat, bnds) float64 3kB -89.75 -88.82 -88.82 ... 88.82 89.75
    lon_bnds   (lon, bnds) float64 6kB -0.4688 0.4688 0.4688 ... 358.6 359.5
    height     float64 8B 2.0
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 480B 2015-01-16T12:00:00 ... 2019-12-16T...
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 18MB 243.3 243.3 243.3 ... 251.1 251.1
    time_bnds  (time, bnds) datetime64[ns] 960B 2015-01-01 ... 2020-01-01
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

In [7]:
!ps -ef | grep cloudify

k204210    52510 4121939  0 10:12 ?        00:00:01 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-c6562e74-6d50-4cf3-92d0-27e4f4cae6f8.json
k204210    54077   52595  0 10:13 ?        00:00:00 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.resource_tracker import main;main(27)
k204210    54079   52595  1 10:13 ?        00:00:27 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=28, pipe_handle=38) --multiprocessing-fork
k204210    54082   52595  1 10:13 ?        00:00:26 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=28, pipe_handle=38) --multiprocessing-fork
k204210    56270 4121939  0 10:18 ?        00:00:07 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime

In [8]:
!kill 52595

In [9]:
import os
os.environ["L_LOSSY"]="1"

In [10]:
%%bash --bg

source activate /work/bm0021/conda-envs/cloudify
python xpublish_example.py \
    example \
    /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/tas/gn/v20190710/*.nc

In [11]:
dslossy=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options,
)

In [12]:
ds5lossy=dslossy.where(ds.time.dt.year.isin(range(2015,2020)),drop=True)
ds5lossy=prepare_for_storage(ds5lossy)

In [13]:
store_ds(ds5,"lossy_compressed.nc")

1.9779608249664307  seconds
512 -rw-r--r-- 1 k204210 bm0021 3.5M Jan  8 10:50 lossy_compressed.nc


In [14]:
import xarray as xr
xr.open_dataset("lossy_compressed.nc").load()

<xarray.Dataset> Size: 18MB
Dimensions:    (lat: 192, bnds: 2, lon: 384, time: 60)
Coordinates:
    lat_bnds   (lat, bnds) float64 3kB -89.75 -88.82 -88.82 ... 88.82 89.75
    lon_bnds   (lon, bnds) float64 6kB -0.4688 0.4688 0.4688 ... 358.6 359.5
    height     float64 8B 2.0
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 480B 2015-01-16T12:00:00 ... 2019-12-16T...
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 18MB 243.3 243.3 243.3 ... 251.1 251.1
    time_bnds  (time, bnds) datetime64[ns] 960B 2015-01-01 ... 2020-01-01
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

We can see that

- uncompressed data is 16MB, compressed is 6MB, lossy compressed is 4MB.
- the accuray is on the second decimal: 243.30435 becomes 243.3125

### Interest in a region of global ESM output stored in records

A "record" refers to the GRIB-record where the entire global field is stored as one binary chunk. A chunk is the most fine-grained level of data access possible.

Especially for high-resolution data, clients may not want to retrieve the full entity of a storage chunk, e.g. an entire global field or an entire month on hourly data, because this results in large data volumes. In such cases, where we know the data access *pattern*, we can adapt the chunk setting of our provided zarr data set to the use case. The cloudify service allows to provide smaller chunk sizes to clients than the orignal storage chunk sizes by providing some server-side computing resources for rechunking and acting as an intermediate layer between data and client. Although the splitting of storage chunks reduces performance on the server, the benefit of bandwidth reduction can be more important.

**Chunk setting**

Best practice is to set the chunks when opening the data. For our test zarr dataset, we split the chunks in both spatial dimensions into half of the original. In each dimension, a chunk needs to cover at least 1 increment i.e. has to be an integer > 0. 

With the `chunks` keyword in the `open_mfdataset` command, it is controlled how the dataset is chunked. These chunks are mapped to *zarr* chunks of the *zarr* API. 

In our example script, we control that chunk setting through specific environment variables:

In [15]:
new_lon_chunk_size=int(len(ds["lon"])/2)
new_lat_chunk_size=int(len(ds["lat"])/2)
print(new_lon_chunk_size, new_lat_chunk_size)

192 96


We set these as environment variables and restart our app which finds these.

In [16]:
os.environ["XPUBLISH_LON_CHUNK_SIZE"]=str(new_lon_chunk_size)
os.environ["XPUBLISH_LAT_CHUNK_SIZE"]=str(new_lat_chunk_size)
#to get back storage chunks:
#del os.environ["XPUBLISH_LON_CHUNK_SIZE"]
#del os.environ["XPUBLISH_LAT_CHUNK_SIZE"]

#kill the existing process

In [47]:
!ps -ef | grep cloudify

k204210    52510 4121939  0 10:12 ?        00:00:01 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-c6562e74-6d50-4cf3-92d0-27e4f4cae6f8.json
k204210    56270 4121939  0 10:18 ?        00:00:07 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-e22a5303-3b28-41cb-b092-ed92a8ff6221.json
k204210    67839 4121939  0 10:42 ?        00:00:13 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-33489dce-534b-42ed-bbe7-6d125e3f6167.json
k204210   178115  177492  0 11:23 ?        00:00:00 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.resource_tracker import main;main(27)
k204210   178117  177492  4 11:23 ?        00:00:05 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.spawn import s

In [48]:
!kill 177492

In [19]:
%%bash --bg
source activate /work/bm0021/conda-envs/cloudify
python xpublish_example.py \
    example \
    /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/tas/gn/v20190710/*.nc

In [20]:
ds2=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options,
    chunks={}
)
ds2

<xarray.Dataset> Size: 314MB
Dimensions:    (lat: 192, time: 1032, bnds: 2, lon: 384)
Coordinates:
    height     float64 8B ...
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 3MB dask.array<chunksize=(60, 96, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 6MB dask.array<chunksize=(60, 192, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 96, 192), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] 17kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

### Towards virtually, highly aggregated datasets that include multiple variables of the same kind

Based on experience, large aggregations are beneficial for data analysis as users can skip finding and merging data sources. E.g. to train an AI model, it helps to simplify the random access within a complete experiment. With cloudify, we can realize a virtual, highly aggregated, dataset that covers the full time series of all variables that share dimensions.

In our example, we can try to concat and merge *all* monthly atmospheric variables more variables. For that, we use a wildcard for variables in the DRS path.

In [40]:
!ps -ef | grep cloudify

k204210    52510 4121939  0 10:12 ?        00:00:01 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-c6562e74-6d50-4cf3-92d0-27e4f4cae6f8.json
k204210    56270 4121939  0 10:18 ?        00:00:07 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-e22a5303-3b28-41cb-b092-ed92a8ff6221.json
k204210    67839 4121939  0 10:42 ?        00:00:12 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-33489dce-534b-42ed-bbe7-6d125e3f6167.json
k204210   178115  177492  0 11:23 ?        00:00:00 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.resource_tracker import main;main(27)
k204210   178117  177492 19 11:23 ?        00:00:03 /work/bm0021/conda-envs/cloudify/bin/python -c from multiprocessing.spawn import s

In [37]:
!kill 142303

In [23]:
#to get back storage chunks:
del os.environ["XPUBLISH_LON_CHUNK_SIZE"]
del os.environ["XPUBLISH_LAT_CHUNK_SIZE"]
del os.environ["L_LOSSY"]

In [38]:
%%bash --bg
source activate /work/bm0021/conda-envs/cloudify
python xpublish_example.py \
    example \
    /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/*/gn/v20190710/*.nc

In [43]:
ds=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options
)

In [44]:
ds

<xarray.Dataset> Size: 144GB
Dimensions:    (time: 1032, lev: 95, bnds: 2, lat: 192, lon: 384, plev: 19)
Coordinates:
    height     float64 8B ...
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lev        (lev) float64 760B 0.9961 0.9826 0.959 ... 2.31e-05 9.816e-06
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * plev       (plev) float64 152B 1e+05 9.25e+04 8.5e+04 ... 1e+03 500.0 100.0
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
Dimensions without coordinates: bnds
Data variables: (12/55)
    ap         (time, lev) float64 784kB dask.array<chunksize=(60, 95), meta=np.ndarray>
    ap_bnds    (time, lev, bnds) float64 2MB dask.array<chunksize=(60, 95, 2), meta=np.ndarray>
    b          (time, lev) float64 784kB dask.array<chunksize=(60, 95), meta=np.ndarray>
    b_bnds     (time, lev, bnds) float64 2MB dask.array<chunksize=(60, 95, 2), meta=np.ndarray>
    cct        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    cl         (time, lev, lat, lon) float32 29GB dask.array<chunksize=(1, 48, 96, 192), meta=np.ndarray>
    ...         ...
    ua         (time, plev, lat, lon) float32 6GB dask.array<chunksize=(1, 10, 96, 192), meta=np.ndarray>
    uas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    va         (time, plev, lat, lon) float32 6GB dask.array<chunksize=(1, 10, 96, 192), meta=np.ndarray>
    vas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    wap        (time, plev, lat, lon) float32 6GB dask.array<chunksize=(1, 10, 96, 192), meta=np.ndarray>
    zg         (time, plev, lat, lon) float32 6GB dask.array<chunksize=(1, 10, 96, 192), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            cl
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/e496e673-c31f-4e03-a4fe-7f09115fed79

In [45]:
ds.nbytes/1024**3

133.80184280127287

In [46]:
ds.isel(time=0,plev=0,lev=0)["tas"].load()#.drop(["height","ap","ap_bnds","b","b_bnds","lev_bnds"]).load()

<xarray.DataArray 'tas' (lat: 192, lon: 384)> Size: 295kB
array([[243.30435, 243.29459, 243.28091, ..., 243.29654, 243.29459,
        243.19107],
       [244.38638, 244.34927, 244.28873, ..., 244.486  , 244.45865,
        244.42935],
       [244.8649 , 244.78677, 244.64224, ..., 245.03287, 244.98795,
        244.9235 ],
       ...,
       [253.50748, 253.49966, 253.52896, ..., 253.38248, 253.42545,
        253.43912],
       [252.84732, 252.85904, 252.861  , ..., 252.63638, 252.75748,
        252.80826],
       [252.24771, 252.23404, 252.28677, ..., 252.23795, 252.23795,
        252.2399 ]], dtype=float32)
Coordinates:
    height   float64 8B 2.0
  * lat      (lat) float64 2kB -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
    lev      float64 8B 0.9961
  * lon      (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
    plev     float64 8B 1e+05
    time     datetime64[ns] 8B 2015-01-16T12:00:00
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    comment:        near-surface (usually, 2 meter) air temperature
    units:          K
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2019-07-20T13:41:51Z altered by CMOR: Treated scalar dime...

We see that

- we host O(100GB) of data "just like that" because of lazy data access.
- the memory usage increases with the number of dask chunks to keep in memory. This is the bottle neck for our data server.
- the more data we aggregate into one dataset, the less endpoints we provide. Thus, the catalog of endpoints becomes small and we use the zarr datasets as the *real* catalogs.
    - this simplifies random access which can be crucial for training of AI models

In the next episode, we use *kerchunks* to avoid repeatedly merging these files when opening and instead use a once-*prepared*  virtual dataset representation of this aggrgeation.